Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [34]:
def evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels):
  num_steps = 3001

  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

### Logstic Regression with L2 loss

In [56]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 20.935780
Minibatch accuracy: 10.2%
Validation accuracy: 11.3%
Minibatch loss at step 500: 3.137359
Minibatch accuracy: 78.9%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 1.669803
Minibatch accuracy: 78.1%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 1.229635
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 0.855276
Minibatch accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss at step 2500: 0.654164
Minibatch accuracy: 83.6%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 0.685812
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Test accuracy: 88.6%


Test accuracy is improved slightly from 85.2% to 88.5% by introducing 0.001 * tf.nn.l2_loss(weights).

### Neural Network with L2 loss

In [37]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h_dim = image_size * image_size * 2
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_dim]))
  b1 = tf.Variable(tf.zeros([h_dim]))
  
  w2 = tf.Variable(
    tf.truncated_normal([h_dim, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, w1) + b1
  logits = tf.nn.relu(logits)
  logits = tf.matmul(logits, w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(w1)
  loss += 0.001 * tf.nn.l2_loss(w2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf_valid_dataset, w1) + b1
  valid_logits = tf.nn.relu(valid_logits)
  valid_logits = tf.matmul(valid_logits, w2) + b2
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf_test_dataset, w1) + b1
  test_logits = tf.nn.relu(test_logits)
  test_logits = tf.matmul(test_logits, w2) + b2
  test_prediction = tf.nn.softmax(test_logits)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 880.933960
Minibatch accuracy: 17.2%
Validation accuracy: 33.8%
Minibatch loss at step 500: 305.560455
Minibatch accuracy: 82.0%
Validation accuracy: 80.9%
Minibatch loss at step 1000: 175.788818
Minibatch accuracy: 85.2%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 105.709923
Minibatch accuracy: 84.4%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 63.912170
Minibatch accuracy: 85.9%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 38.314262
Minibatch accuracy: 90.6%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 23.414917
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Test accuracy: 93.4%


The accuracy is improved on Neural Network model as well from 88.9% to 93.1%.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### Neural Network Model with Batch Size 4 and L2 Loss.

In [55]:
batch_size = 4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h_dim = image_size * image_size * 2
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_dim]))
  b1 = tf.Variable(tf.zeros([h_dim]))
  
  w2 = tf.Variable(
    tf.truncated_normal([h_dim, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, w1) + b1
  logits = tf.nn.relu(logits)
  logits = tf.matmul(logits, w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(w1)
  loss += 0.001 * tf.nn.l2_loss(w2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf_valid_dataset, w1) + b1
  valid_logits = tf.nn.relu(valid_logits)
  valid_logits = tf.matmul(valid_logits, w2) + b2
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf_test_dataset, w1) + b1
  test_logits = tf.nn.relu(test_logits)
  test_logits = tf.matmul(test_logits, w2) + b2
  test_prediction = tf.nn.softmax(test_logits)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 835.662842
Minibatch accuracy: 25.0%
Validation accuracy: 16.0%
Minibatch loss at step 500: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 50.0%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Test accuracy: 10.0%


When I reduce the batch size to 4, the loss value gets extremely high. Small batch size make the minibatch accuracy vibrates.

### Neural Network Model with Batch Size 8 and L2 Loss.

In [45]:
batch_size = 8

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h_dim = image_size * image_size * 2
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_dim]))
  b1 = tf.Variable(tf.zeros([h_dim]))
  
  w2 = tf.Variable(
    tf.truncated_normal([h_dim, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, w1) + b1
  logits = tf.nn.relu(logits)
  logits = tf.matmul(logits, w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(w1)
  loss += 0.001 * tf.nn.l2_loss(w2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf_valid_dataset, w1) + b1
  valid_logits = tf.nn.relu(valid_logits)
  valid_logits = tf.matmul(valid_logits, w2) + b2
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf_test_dataset, w1) + b1
  test_logits = tf.nn.relu(test_logits)
  test_logits = tf.matmul(test_logits, w2) + b2
  test_prediction = tf.nn.softmax(test_logits)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 828.111694
Minibatch accuracy: 12.5%
Validation accuracy: 21.3%
Minibatch loss at step 500: 2204157952.000000
Minibatch accuracy: 25.0%
Validation accuracy: 41.3%
Minibatch loss at step 1000: 25229770358784.000000
Minibatch accuracy: 37.5%
Validation accuracy: 30.7%
Minibatch loss at step 1500: 24921382378799104.000000
Minibatch accuracy: 25.0%
Validation accuracy: 32.5%
Minibatch loss at step 2000: 255363422730510663680.000000
Minibatch accuracy: 37.5%
Validation accuracy: 42.0%
Minibatch loss at step 2500: 692270780692458773676032.000000
Minibatch accuracy: 25.0%
Validation accuracy: 28.2%
Minibatch loss at step 3000: 7475205091519365433398394880.000000
Minibatch accuracy: 50.0%
Validation accuracy: 32.9%
Test accuracy: 35.8%


Ramped up the batch size to 8. The loss value is calculable, but the accuracy remains low. There's a gap between train accuracy and validation accuracy.

### Neural Network Model with Batch Size 16 and L2 Loss.

In [53]:
batch_size = 16

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h_dim = image_size * image_size * 2
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_dim]))
  b1 = tf.Variable(tf.zeros([h_dim]))
  
  w2 = tf.Variable(
    tf.truncated_normal([h_dim, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, w1) + b1
  logits = tf.nn.relu(logits)
  logits = tf.matmul(logits, w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(w1)
  loss += 0.001 * tf.nn.l2_loss(w2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf_valid_dataset, w1) + b1
  valid_logits = tf.nn.relu(valid_logits)
  valid_logits = tf.matmul(valid_logits, w2) + b2
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf_test_dataset, w1) + b1
  test_logits = tf.nn.relu(test_logits)
  test_logits = tf.matmul(test_logits, w2) + b2
  test_prediction = tf.nn.softmax(test_logits)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 972.694885
Minibatch accuracy: 6.2%
Validation accuracy: 26.1%
Minibatch loss at step 500: 480.078125
Minibatch accuracy: 87.5%
Validation accuracy: 71.8%
Minibatch loss at step 1000: 255.639679
Minibatch accuracy: 75.0%
Validation accuracy: 62.1%
Minibatch loss at step 1500: 153.255341
Minibatch accuracy: 62.5%
Validation accuracy: 64.6%
Minibatch loss at step 2000: 85.175552
Minibatch accuracy: 68.8%
Validation accuracy: 72.5%
Minibatch loss at step 2500: 51.245132
Minibatch accuracy: 87.5%
Validation accuracy: 76.0%
Minibatch loss at step 3000: 31.350275
Minibatch accuracy: 87.5%
Validation accuracy: 77.7%
Test accuracy: 85.5%


If I set the batch size as 16, it can be trained almost normally.

### Neural Network Model with Batch Size 8 and no L2 Loss.

In [58]:
batch_size = 8

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h_dim = image_size * image_size * 2
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_dim]))
  b1 = tf.Variable(tf.zeros([h_dim]))
  
  w2 = tf.Variable(
    tf.truncated_normal([h_dim, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, w1) + b1
  logits = tf.nn.relu(logits)
  logits = tf.matmul(logits, w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf_valid_dataset, w1) + b1
  valid_logits = tf.nn.relu(valid_logits)
  valid_logits = tf.matmul(valid_logits, w2) + b2
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf_test_dataset, w1) + b1
  test_logits = tf.nn.relu(test_logits)
  test_logits = tf.matmul(test_logits, w2) + b2
  test_prediction = tf.nn.softmax(test_logits)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 547.158386
Minibatch accuracy: 0.0%
Validation accuracy: 19.6%
Minibatch loss at step 500: 6215875584.000000
Minibatch accuracy: 37.5%
Validation accuracy: 38.9%
Minibatch loss at step 1000: 658151527415808.000000
Minibatch accuracy: 25.0%
Validation accuracy: 42.4%
Minibatch loss at step 1500: 1795249600483819520.000000
Minibatch accuracy: 12.5%
Validation accuracy: 31.4%
Minibatch loss at step 2000: 1005269873930797676756992.000000
Minibatch accuracy: 50.0%
Validation accuracy: 41.2%
Minibatch loss at step 2500: 10488907224682804854705356800.000000
Minibatch accuracy: 12.5%
Validation accuracy: 37.5%
Minibatch loss at step 3000: 48770354850904851786045683924992.000000
Minibatch accuracy: 25.0%
Validation accuracy: 43.8%
Test accuracy: 48.3%


### Logistic Regression Model with Batch Size 8

In [61]:
batch_size = 2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 13.341852
Minibatch accuracy: 50.0%
Validation accuracy: 13.4%
Minibatch loss at step 500: 32.974693
Minibatch accuracy: 50.0%
Validation accuracy: 67.8%
Minibatch loss at step 1000: 2.660239
Minibatch accuracy: 100.0%
Validation accuracy: 70.9%
Minibatch loss at step 1500: 2.408784
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 2000: 2.123155
Minibatch accuracy: 100.0%
Validation accuracy: 72.6%
Minibatch loss at step 2500: 14.659731
Minibatch accuracy: 50.0%
Validation accuracy: 71.4%
Minibatch loss at step 3000: 1.942750
Minibatch accuracy: 100.0%
Validation accuracy: 67.5%
Test accuracy: 75.2%


Huge gap between training accuracy and test accuracy.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### Adding Dropout on Neural Network model with large keeping probability.

In [51]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h_dim = image_size * image_size * 2
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_dim]))
  b1 = tf.Variable(tf.zeros([h_dim]))
  
  w2 = tf.Variable(
    tf.truncated_normal([h_dim, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  keep_prob = 0.8
  logits = tf.matmul(tf.nn.dropout(tf_train_dataset, keep_prob), w1) + b1
  logits = tf.nn.relu(logits)
  logits = tf.matmul(logits, w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(w1)
  loss += 0.001 * tf.nn.l2_loss(w2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf_valid_dataset, w1) + b1
  valid_logits = tf.nn.relu(valid_logits)
  valid_logits = tf.matmul(valid_logits, w2) + b2
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf_test_dataset, w1) + b1
  test_logits = tf.nn.relu(test_logits)
  test_logits = tf.matmul(test_logits, w2) + b2
  test_prediction = tf.nn.softmax(test_logits)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 977.635620
Minibatch accuracy: 3.9%
Validation accuracy: 33.1%
Minibatch loss at step 500: 314.817810
Minibatch accuracy: 76.6%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 175.912827
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 105.694519
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 62.812805
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 37.817158
Minibatch accuracy: 84.4%
Validation accuracy: 84.0%
Minibatch loss at step 3000: 23.040064
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Test accuracy: 91.6%


### Adding Dropout on Neural Network model with small keeping probability.

In [62]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h_dim = image_size * image_size * 2
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_dim]))
  b1 = tf.Variable(tf.zeros([h_dim]))
  
  w2 = tf.Variable(
    tf.truncated_normal([h_dim, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  keep_prob = 0.5
  logits = tf.matmul(tf.nn.dropout(tf_train_dataset, keep_prob), w1) + b1
  logits = tf.nn.relu(logits)
  logits = tf.matmul(logits, w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(w1)
  loss += 0.001 * tf.nn.l2_loss(w2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf_valid_dataset, w1) + b1
  valid_logits = tf.nn.relu(valid_logits)
  valid_logits = tf.matmul(valid_logits, w2) + b2
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf_test_dataset, w1) + b1
  test_logits = tf.nn.relu(test_logits)
  test_logits = tf.matmul(test_logits, w2) + b2
  test_prediction = tf.nn.softmax(test_logits)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 1167.873291
Minibatch accuracy: 8.6%
Validation accuracy: 36.1%
Minibatch loss at step 500: 338.161804
Minibatch accuracy: 78.1%
Validation accuracy: 78.5%
Minibatch loss at step 1000: 178.389053
Minibatch accuracy: 75.8%
Validation accuracy: 78.7%
Minibatch loss at step 1500: 106.918610
Minibatch accuracy: 74.2%
Validation accuracy: 78.4%
Minibatch loss at step 2000: 61.764938
Minibatch accuracy: 78.1%
Validation accuracy: 78.4%
Minibatch loss at step 2500: 37.225433
Minibatch accuracy: 73.4%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 21.875019
Minibatch accuracy: 83.6%
Validation accuracy: 82.7%
Test accuracy: 89.5%


### Adding Dropout with small batch model.

In [63]:
batch_size = 8

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h_dim = image_size * image_size * 2
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_dim]))
  b1 = tf.Variable(tf.zeros([h_dim]))
  
  w2 = tf.Variable(
    tf.truncated_normal([h_dim, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  keep_prob = 0.8
  logits = tf.matmul(tf.nn.dropout(tf_train_dataset, keep_prob), w1) + b1
  logits = tf.nn.relu(logits)
  logits = tf.matmul(logits, w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(w1)
  loss += 0.001 * tf.nn.l2_loss(w2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf_valid_dataset, w1) + b1
  valid_logits = tf.nn.relu(valid_logits)
  valid_logits = tf.matmul(valid_logits, w2) + b2
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf_test_dataset, w1) + b1
  test_logits = tf.nn.relu(test_logits)
  test_logits = tf.matmul(test_logits, w2) + b2
  test_prediction = tf.nn.softmax(test_logits)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 1004.980164
Minibatch accuracy: 12.5%
Validation accuracy: 21.4%
Minibatch loss at step 500: 10495202304.000000
Minibatch accuracy: 37.5%
Validation accuracy: 41.5%
Minibatch loss at step 1000: 3205322281844736.000000
Minibatch accuracy: 37.5%
Validation accuracy: 36.4%
Minibatch loss at step 1500: 28196184045943521280.000000
Minibatch accuracy: 50.0%
Validation accuracy: 43.1%
Minibatch loss at step 2000: 5811620203227588279140352.000000
Minibatch accuracy: 50.0%
Validation accuracy: 36.3%
Minibatch loss at step 2500: 204345543536495822979561684992.000000
Minibatch accuracy: 12.5%
Validation accuracy: 26.8%
Minibatch loss at step 3000: 2199032584172649978658510990213120.000000
Minibatch accuracy: 25.0%
Validation accuracy: 38.0%
Test accuracy: 42.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### Learning rate decay

In [72]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h_dim = image_size * image_size * 2
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_dim]))
  b1 = tf.Variable(tf.zeros([h_dim]))
  
  w2 = tf.Variable(
    tf.truncated_normal([h_dim, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  keep_prob = 0.90
  logits = tf.matmul(tf.nn.dropout(tf_train_dataset, keep_prob), w1) + b1
  #logits = tf.matmul(tf_train_dataset, w1) + b1
  logits = tf.nn.relu(logits)
  logits = tf.matmul(logits, w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.001 * tf.nn.l2_loss(w1)
  loss += 0.001 * tf.nn.l2_loss(w2)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps=10000, decay_rate=0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(
    loss, global_step=global_step)

  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf_valid_dataset, w1) + b1
  valid_logits = tf.nn.relu(valid_logits)
  valid_logits = tf.matmul(valid_logits, w2) + b2
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf_test_dataset, w1) + b1
  test_logits = tf.nn.relu(test_logits)
  test_logits = tf.matmul(test_logits, w2) + b2
  test_prediction = tf.nn.softmax(test_logits)
evaluate(graph, batch_size, train_dataset, train_labels, train_prediction,
             valid_prediction, valid_labels, test_prediction, test_labels)

Initialized
Minibatch loss at step 0: 1062.656006
Minibatch accuracy: 6.2%
Validation accuracy: 34.2%
Minibatch loss at step 500: 304.320404
Minibatch accuracy: 79.7%
Validation accuracy: 80.9%
Minibatch loss at step 1000: 175.730637
Minibatch accuracy: 83.6%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 105.907669
Minibatch accuracy: 77.3%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 63.359344
Minibatch accuracy: 83.6%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 38.143074
Minibatch accuracy: 86.7%
Validation accuracy: 85.0%
Minibatch loss at step 3000: 23.263596
Minibatch accuracy: 85.9%
Validation accuracy: 86.2%
Test accuracy: 92.4%
